In [102]:
#import the libraries
import pandas as pd
import scipy 
import numpy as np
from scipy import stats 
import statsmodels.api as sm
from plotly import figure_factory as FF
from statsmodels.formula.api import ols
from scipy.stats import chi2_contingency

ModuleNotFoundError: No module named 'plotly'

In [8]:
#Minitab File : Cutlets.csv
cutletsdf= pd.read_csv('Cutlets.csv')
cutletsdf.head(5)

,Unit A,Unit B
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


In [5]:
cutletsdf.describe()

,Unit A,Unit B
count,35.000000,35.000000
mean,7.019091,6.964297
std,0.288408,0.343401
min,6.437600,6.038000
25%,6.831500,6.753600
50%,6.943800,6.939900
75%,7.280550,7.195000
max,7.516900,7.545900


In [7]:
cutletsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Unit A  35 non-null     float64
 1   Unit B  35 non-null     float64
dtypes: float64(2)
memory usage: 688.0 bytes


#### Q1)      A F&B manager wants to determine whether there is any significant difference in the diameter of the cutlet between two units. A randomly selected sample of cutlets was collected from both units and measured? Analyze the data and draw inferences at 5% significance level. Please state the assumptions and tests that you carried out to check validity of the assumptions.


## 2 Sample t-test

H0: μ1=μ2 No difference in the diameter.

H1: μ1!=μ2 Difference in the diameter.

In [32]:
#Hypothesis testing
sm.stats.ttest_ind(cutletsdf[['Unit A']], cutletsdf[['Unit B']])

(array([0.72286887]), array([0.47223947]), 68.0)

The test result shows the pvalue(0.47223947) is greater than significance level (0.05).
Which means there is 47% chance we'd see sample data this far apart if the two groups tested are actually identical.
At 95% confidence level we would fail to reject the null hypothesis, since the p-value is greater than the significance level of 5%.
Hence, there is no difference in the diameter of the cutlet between two units.

## Q2) A hospital wants to determine whether there is any difference in the average Turn Around Time (TAT) of reports of the laboratories on their preferred list. They collected a random sample and recorded TAT for reports of 4 laboratories. TAT is defined as sample collected to report dispatch.
#### Analyze the data and determine whether there is any difference in average TAT among the different laboratories at 5% significance level.

In [9]:
#Minitab File: LabTAT.csv
labtatdf= pd.read_csv('LabTAT.csv')
labtatdf.head(5)

,Laboratory 1,Laboratory 2,Laboratory 3,Laboratory 4
0,185.35,165.53,176.70,166.13
1,170.49,185.91,198.45,160.79
2,192.77,194.92,201.23,185.18
3,177.33,183.00,199.61,176.42
4,193.41,169.57,204.63,152.60


In [11]:
labtatdf.describe(include='all')

,Laboratory 1,Laboratory 2,Laboratory 3,Laboratory 4
count,120.000000,120.000000,120.000000,120.00000
mean,178.361583,178.902917,199.913250,163.68275
std,13.173594,14.957114,16.539033,15.08508
min,138.300000,140.550000,159.690000,124.06000
25%,170.335000,168.025000,188.232500,154.05000
50%,178.530000,178.870000,199.805000,164.42500
75%,186.535000,189.112500,211.332500,172.88250
max,216.390000,217.860000,238.700000,205.18000


In [19]:
labtatdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Laboratory 1  120 non-null    float64
 1   Laboratory 2  120 non-null    float64
 2   Laboratory 3  120 non-null    float64
 3   Laboratory 4  120 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB


In [26]:
labtatdf.columns='Laboratory1','Laboratory2','Laboratory3','Laboratory4'
labtatdf.head(5)

,Laboratory1,Laboratory2,Laboratory3,Laboratory4
0,185.35,165.53,176.70,166.13
1,170.49,185.91,198.45,160.79
2,192.77,194.92,201.23,185.18
3,177.33,183.00,199.61,176.42
4,193.41,169.57,204.63,152.60


## One-Way ANOVA

H0: μ1=μ2=μ3=μ4, The means of the samples are equal. There is no difference in the average Turn Around Time (TAT) of reports of the laboratories.

H1: μ1!=μ2!=μ3!=μ4, one or more of the means of the samples are unequal. There is difference in the average Turn Around Time (TAT) of reports of the laboratories.     

In [28]:
##########Normality Test ############

print(stats.shapiro(labtatdf.Laboratory1))    #Shapiro Test
print(stats.shapiro(labtatdf.Laboratory2))
print(stats.shapiro(labtatdf.Laboratory3))
print(stats.shapiro(labtatdf.Laboratory4))

############## Variance test #########
print(scipy.stats.levene(labtatdf.Laboratory1, labtatdf.Laboratory2))
print(scipy.stats.levene(labtatdf.Laboratory2, labtatdf.Laboratory3))
print(scipy.stats.levene(labtatdf.Laboratory3, labtatdf.Laboratory4))
print(scipy.stats.levene(labtatdf.Laboratory4, labtatdf.Laboratory1))

ShapiroResult(statistic=0.9901824593544006, pvalue=0.5506953597068787)
ShapiroResult(statistic=0.9936322569847107, pvalue=0.8637524843215942)
ShapiroResult(statistic=0.9886345267295837, pvalue=0.4205053448677063)
ShapiroResult(statistic=0.9913753271102905, pvalue=0.6618951559066772)
LeveneResult(statistic=3.5495027780905763, pvalue=0.06078228171776711)
LeveneResult(statistic=0.9441465124387124, pvalue=0.33220021420602397)
LeveneResult(statistic=2.037958464521512, pvalue=0.15472618294425391)
LeveneResult(statistic=1.5000140718506723, pvalue=0.22188001348277267)


In [29]:
############# One - Way Anova###################
mod=ols('Laboratory1 ~ (Laboratory2 + Laboratory3 + Laboratory4)',data=labtatdf).fit()

aov_table=sm.stats.anova_lm(mod)

print(aov_table)
stats.f_oneway(labtatdf.Laboratory1,labtatdf.Laboratory2,labtatdf.Laboratory3,labtatdf.Laboratory4 )

                df        sum_sq     mean_sq         F    PR(>F)
Laboratory2    1.0    332.030416  332.030416  1.940311  0.166299
Laboratory3    1.0    203.853111  203.853111  1.191271  0.277335
Laboratory4    1.0    265.614707  265.614707  1.552192  0.215323
Residual     116.0  19850.186366  171.122296       NaN       NaN


F_onewayResult(statistic=118.70421654401437, pvalue=2.1156708949992414e-57)

The test output yields an F-statistic of 118.704 and a p-value of 2.115670, indicating that there is no significant difference between the means of each group.
Hence, p-value is greater than significance level of (5%).The means of the samples are equal. There is no difference in the average Turn Around Time (TAT) of reports of the laboratories.

## Q3) Sales of products in four different regions is tabulated for males and females. Find if male-female buyer rations are similar across regions.

In [37]:
# Buyer Ratio.csv
buyerratiodf= pd.read_csv('BuyerRatio.csv',index_col=0)
buyerratiodf.head(5)

,East,West,North,South
Observed Values,,,,
Males,50,142,131,70
Females,435,1523,1356,750


In [38]:
buyerratiodf.describe(include='all')

,East,West,North,South
count,2.000000,2.000000,2.000000,2.000000
mean,242.500000,832.500000,743.500000,410.000000
std,272.236111,976.514465,866.205807,480.832611
min,50.000000,142.000000,131.000000,70.000000
25%,146.250000,487.250000,437.250000,240.000000
50%,242.500000,832.500000,743.500000,410.000000
75%,338.750000,1177.750000,1049.750000,580.000000
max,435.000000,1523.000000,1356.000000,750.000000


In [39]:
buyerratiodf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, Males to Females
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   East    2 non-null      int64
 1   West    2 non-null      int64
 2   North   2 non-null      int64
 3   South   2 non-null      int64
dtypes: int64(4)
memory usage: 80.0+ bytes


## Chi-Squared Tests

H0: Male-female buyer rations are similar across regions.

H1: Male-female buyer rations are not same across regions.

In [103]:
################Chi-Square Test ################

stats.chi2_contingency(buyerratiodf)
# output(chi2,p,df,expected)

(1.595945538661058,
 0.6603094907091882,
 3,
 array([[  42.76531299,  146.81287862,  131.11756787,   72.30424052],
        [ 442.23468701, 1518.18712138, 1355.88243213,  747.69575948]]))

p-value is greater than significance level of 5%. We fail to reject Null hypothesis.

Hence, Male-female buyer rations are similar across regions.

## Q4) TeleCall uses 4 centers around the globe to process customer order forms. They audit a certain %  of the customer order forms. Any error in order form renders it defective and has to be reworked before processing.  The manager wants to check whether the defective %  varies by centre. Please analyze the data at 5% significance level and help the manager draw appropriate inferences.

In [84]:
# CustomerOrderForm.csv
customerorderformdf= pd.read_csv('Costomer+OrderForm.csv')
customerorderformdf.head(5)

,Phillippines,Indonesia,Malta,India
0,Error Free,Error Free,Defective,Error Free
1,Error Free,Error Free,Error Free,Defective
2,Error Free,Defective,Defective,Error Free
3,Error Free,Error Free,Error Free,Error Free
4,Error Free,Error Free,Defective,Error Free


In [85]:
customerorderformdf.describe(include='all')

,Phillippines,Indonesia,Malta,India
count,300,300,300,300
unique,2,2,2,2
top,Error Free,Error Free,Error Free,Error Free
freq,271,267,269,280


## Chi-Squared Tests

The chi-squared test of independence tests whether categorical variables are independent. 

H0: Independence of categorical variables customer order forms defective% does not varies by centre.

H1: Dependence of categorical variables customer order forms defective% varies by centre.    

In [88]:
ph=customerorderformdf.Phillippines.value_counts()

Error Free    271
Defective      29
Name: Phillippines, dtype: int64

In [89]:
ind=customerorderformdf.Indonesia.value_counts()
ind

Error Free    267
Defective      33
Name: Indonesia, dtype: int64

In [90]:
ma=customerorderformdf.Malta.value_counts()
ma

Error Free    269
Defective      31
Name: Malta, dtype: int64

In [92]:
india=customerorderformdf.India.value_counts()
india

Error Free    280
Defective      20
Name: India, dtype: int64

In [101]:
data= [[271,267,269,280],[29,33,31,20]]
data

[[271, 267, 269, 280], [29, 33, 31, 20]]

In [104]:
################Chi-Square Test ################

stats.chi2_contingency(data)
# output(chi2,p,df,expected)

(3.858960685820355,
 0.2771020991233135,
 3,
 array([[271.75, 271.75, 271.75, 271.75],
        [ 28.25,  28.25,  28.25,  28.25]]))

As, on assumption of the significance level of 5%, the p-value will be greater.
Therefore, Null hypothesis is accepted that is independence of categorical variables customer order forms defective% does not varies by centre.  

The output shows the chi-square statistic, the p-value and the degrees of freedom followed by the expected counts.

Given the high p-value, the test result does not detect a significant relationship between the variables.